# Classificador de Twitter

### Autor: Ricardo Robson ~rrms at cin.ufpe.br
### Baseado em: http://minerandodados.com.br/index.php/2017/03/15/analise-de-sentimentos-twitter-como-fazer/

O objetivo deste notebook é construir um classificador de sentimento de tweets, utilizando o Algoritmo Naive Bayes, classificando textos de tweets em Positivo ou Negativo.

Faremos uso de um cospus contendo pouco mais de 2900 tweets já classificados.

In [10]:
#Imports que serão usados ao longo da construção e análise do classificador
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

In [11]:
#Lendo o corpus de tweets classificados
dataset = pd.read_csv('twitter_corpus.csv')
dataset.count()

Text         2997
Sentiment    2997
dtype: int64

In [14]:
#Separando textos e classes (sentimentos)
tweets = dataset['Text'].values
classes = dataset['Sentiment'].values

In [7]:
#Criamos um objeto do tipo CountVectorizer chamado vectorizer.
vectorizer = CountVectorizer(analyzer="word")

#Usamos o objeto vectorizer para calcular a frequência de todas as palavras 
#da lista de tweets e armazenamos seu retorno em freq_tweets.
freq_tweets = vectorizer.fit_transform(tweets)

#Criamos um objeto chamado modelo do tipo Naive Bayes Multinomial.
#Esse é o algoritmo de classificação que iremos utilizar.
modelo = MultinomialNB()

#Por fim treinamos o modelo usando a frequência de palavras (freq_tweets)
#e as classes de cada instância.
modelo.fit(freq_tweets,classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [18]:
#Criando uma pequena lista para testes
testes = ['Feeeling like shit right now. I really want to sleep, but nooo I have 3 hours of dancing and an art assignment to finish.',
         'goodbye exams, HELLO ALCOHOL TONIGHT',
         'trae is so sweet! He just bought me a new baithing suit!! Wove him']
#Testando
freq_testes = vectorizer.transform(testes)
modelo.predict(freq_testes)

array(['neg', 'pos', 'pos'],
      dtype='<U3')

array(['neg', 'pos', 'pos'],
      dtype='<U3')